In [ ]:
import json
from args import args_define
from typing import List, Tuple, Dict
import clip
import open_clip
import numpy as np
import torch
from datasets import CIRRDataset, CIRCODataset
from utils import extract_image_features, device, collate_fn, PROJECT_ROOT, targetpad_transform
import torch
import torch.nn.functional as F


clip_model, _, clip_preprocess = open_clip.create_model_and_transforms('ViT-g-14', device=device, pretrained='laion2b_s34b_b88k')
tokenizer = open_clip.get_tokenizer('ViT-g-14')
args = args_define.args 

####初始化检索图片特征
classic_test_dataset = CIRRDataset(args.dataset_path, 'test1', 'classic', clip_preprocess)
index_features, index_names, _ = extract_image_features(classic_test_dataset, clip_model)
index_features = index_features.to(device)
index_features = F.normalize(index_features, dim=-1).float()

###初始化参考文本特征
reference_names = "test1-1051-1-img1"
text = ["husky puppy, wooden floor, towels.","husky puppy, wooden floor, towels.","husky puppy, wooden floor, towels."]
text_features_list = []
for cap in text:
    tokenized_input_captions = tokenizer(cap, context_length=77).to(device)
    text_features = clip_model.encode_text(tokenized_input_captions)
    text_features_list.append(text_features)
text_features_list = torch.vstack(text_features_list)
text_features = torch.mean(text_features_list, dim=0)
predicted_features = F.normalize(text_features).float()

#### 计算相似度
distances = 1 - predicted_features @ index_features.T
sorted_indices = torch.argsort(distances, dim=-1).cpu()
sorted_index_names = np.array(index_names)[sorted_indices]


# Delete the reference image from the results
reference_mask = torch.tensor(
        sorted_index_names != np.repeat(np.array(reference_names), len(index_names)).reshape(len(sorted_index_names),
                                                                                             -1))
sorted_index_names = sorted_index_names[reference_mask].reshape(sorted_index_names.shape[0],
                                                                    sorted_index_names.shape[1] - 1)
sorted_index_names = sorted_index_names.tolist()
print(sorted_index_names)


(tensor([0.1305, 2.2971, 3.2298]),
 tensor([[0.1305, 0.0079, 0.1849],
         [0.5490, 2.2971, 0.9173],
         [3.7014, 3.2826, 3.2298]]))

In [16]:
import torch

# 假设 a 是一个形状为 (20, 2) 的张量
a = torch.randn(20, 2)

# 计算每一列的平均值
column_means = torch.mean(a, dim=0)

print(column_means)

tensor([-0.1267, -0.1935])


In [20]:
import random
import clip
from transformers import BertTokenizer, BertModel
import torch
from PIL import Image


device = torch.device("cuda")
# 初始化BERT编码器和分词器
clip_model,_ = clip.load('ViT-L/14', device=device, jit=False)
tokenizer = BertTokenizer.from_pretrained('bert-base-uncased')
# model = BertModel.from_pretrained('bert-base-uncased')

# 输入文本
text = "The image shows a group of wolves playing in the snow, with a wooden wall in the background. The wolves appear to be engaged in a fierce battle, with one of them lying on the ground and the others surrounding it."

# 将文本随机遮掩部分词语
def mask_text(text, mask_token='[MASK]', mask_prob=0):
    tokens = tokenizer.tokenize(text)
    masked_tokens = [
        (mask_token if random.random() < mask_prob else token)
        for token in tokens
    ]
    masked_text = ' '.join(masked_tokens)
    return masked_text

# 生成遮掩后的文本
masked_text = mask_text(text)
print("Masked Text:", masked_text)

# # 将遮掩后的文本编码
tokenized_input_captions = tokenizer(masked_text, context_length=77).to(device)
text_features = clip_model.encode_text(tokenized_input_captions)

# # 图像特征获取（假设有一个预训练的图像编码器）
reference_images = Image.open('../CIRR/test1/test1-147-1-img1.png')
image_features = clip_model.encode_image(reference_images)

similarity = (text_features @ image_features.T)
print(similarity)



# # 获取文本特征
# text_features = outputs.last_hidden_state

# # 图像特征获取（假设有一个预训练的图像编码器）
# # image_features = image_encoder(image_input)

# # 匹配步骤可以通过计算相似度来完成，如余弦相似度或其他对比损失


Keyword arguments {'context_length': 77} not recognized.


Masked Text: the image shows a group of wolves playing in the snow , with a wooden wall in the background . the wolves appear to be engaged in a fierce battle , with one of them lying on the ground and the others surrounding it .


TypeError: embedding(): argument 'indices' (position 2) must be Tensor, not BatchEncoding